In [ ]:
import os
# Find the latest version of spark 3.x  from https://downloads.apache.org/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.1'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

In [ ]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("demographicsFilter").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/11/07 11:58:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/11/07 11:58:06 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/1/demographics.csv"
spark.sparkContext.addFile(url)
df = spark.read.option('header', 'true').csv(SparkFiles.get("demographics.csv"), inferSchema=True, sep=',')

# Show DataFrame
df.show()


[Stage 0:>                                                          (0 + 1) / 1]



+---+-----------------+---+--------+---------+--------+--------------------+---------------+------+--------------------+
| id|             name|age|height_m|weight_kg|children|          occupation|academic_degree|salary|            location|
+---+-----------------+---+--------+---------+--------+--------------------+---------------+------+--------------------+
|  1|    Glad Gavrieli| 38|    1.52|       74|       0|Computer Systems ...|       Bachelor|    78|           Louisiana|
|  2|  Henrieta Fittes| 34|    1.72|       39|       4|             Teacher|         Master|    44|            Illinois|
|  3|   Peyton Dulanty| 24|     1.8|       47|       5|Senior Quality En...|            PhD|    44|      North Carolina|
|  4|     Denna Morgen| 48|    1.81|       71|       5|   Account Executive|         Master|    81|          California|
|  5|    Camella Izaks| 34|    1.65|       60|       1|   Director of Sales|            PhD|    76|                Ohio|
|  6|     Shara Esposi| 49|     

In [ ]:
# What occupation had the highest salary?


+-----------------+------+
|       occupation|Salary|
+-----------------+------+
|Chemical Engineer|   120|
+-----------------+------+



In [ ]:
# What occupation had the lowest salary?


+----------------+------+
|      occupation|Salary|
+----------------+------+
|Product Engineer|    40|
+----------------+------+



In [ ]:
# What is the mean salary of this dataset?
from pyspark.sql.functions import avg


+-----------+
|avg(Salary)|
+-----------+
|     79.475|
+-----------+



In [ ]:
# What is the max and min of the Salary column?
from pyspark.sql.functions import max, min


+-----------+-----------+
|max(Salary)|min(Salary)|
+-----------+-----------+
|        120|         40|
+-----------+-----------+



In [ ]:
# Show all of the occupations where salaries were above 80k
from pyspark.sql.functions import count


+--------------------+
|          occupation|
+--------------------+
|   Account Executive|
|Sales Representative|
|        Developer II|
|     Design Engineer|
|     Project Manager|
|       Programmer II|
|Sales Representative|
|Physical Therapy ...|
|Payment Adjustmen...|
|Mechanical System...|
|     Media Manager I|
|   Account Executive|
|           Professor|
|Community Outreac...|
| Clinical Specialist|
|Human Resources A...|
|Nuclear Power Eng...|
|      Civil Engineer|
|Human Resources M...|
|Senior Cost Accou...|
+--------------------+
only showing top 20 rows



In [ ]:
# BONUS
# What is the average age and height for each academic degree type?
# HINT: You will need to use `groupby` to solve this


+---------------+------------------+------------------+
|academic_degree|          avg(age)|     avg(height_m)|
+---------------+------------------+------------------+
|            PhD| 42.87818696883853|1.7537393767705372|
|         Master|42.105095541401276|1.7606050955414014|
|       Bachelor| 43.85585585585586|1.7371771771771771|
+---------------+------------------+------------------+

